In [1]:
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras import backend as K

import tensorflow as tf

Using TensorFlow backend.


In [5]:
EPISODES = 50

In [3]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.99
        self.learning_rate = 0.001
        self.model = self._build_model()
        self.target_model = self._build_model()
        self.update_target_model()

    """Huber loss for Q Learning
    References: https://en.wikipedia.org/wiki/Huber_loss
                https://www.tensorflow.org/api_docs/python/tf/losses/huber_loss
    """

    def _huber_loss(self, y_true, y_pred, clip_delta=1.0):
        error = y_true - y_pred
        cond  = K.abs(error) <= clip_delta

        squared_loss = 0.5 * K.square(error)
        quadratic_loss = 0.5 * K.square(clip_delta) + clip_delta * (K.abs(error) - clip_delta)

        return K.mean(tf.where(cond, squared_loss, quadratic_loss))

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss=self._huber_loss,
                      optimizer=Adam(lr=self.learning_rate))
        return model

    def update_target_model(self):
        # copy weights from model to target_model
        self.target_model.set_weights(self.model.get_weights())

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = self.model.predict(state)
            if done:
                target[0][action] = reward
            else:
                # a = self.model.predict(next_state)[0]
                t = self.target_model.predict(next_state)[0]
                target[0][action] = reward + self.gamma * np.amax(t)
                # target[0][action] = reward + self.gamma * t[np.argmax(a)]
            self.model.fit(state, target, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)


In [8]:
if __name__ == "__main__":
    env = gym.make('CartPole-v1')
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n
    agent = DQNAgent(state_size, action_size)
    # agent.load("./save/cartpole-ddqn.h5")
    done = False
    batch_size = 32

    for e in range(EPISODES):
        state = env.reset()
        state = np.reshape(state, [1, state_size])
        for time in range(500):
            # env.render()
            action = agent.act(state)
            print("action ->",action, state)
            next_state, reward, done, _ = env.step(action)
            reward = reward if not done else -10
            next_state = np.reshape(next_state, [1, state_size])
            agent.remember(state, action, reward, next_state, done)
            state = next_state
            if done:
                agent.update_target_model()
                print("episode: {}/{}, score: {}, e: {:.2}"
                      .format(e, EPISODES, time, agent.epsilon))
                break
            if len(agent.memory) > batch_size:
                agent.replay(batch_size)
        # if e % 10 == 0:
        #     agent.save("./save/cartpole-ddqn.h5")


action -> 1 [[-0.01399115 -0.02170298 -0.00243015 -0.02845564]]
action -> 0 [[-0.01442521  0.17345374 -0.00299926 -0.32190432]]
action -> 0 [[-0.01095613 -0.02162538 -0.00943735 -0.03016875]]
action -> 0 [[-0.01138864 -0.21661073 -0.01004073  0.25952172]]
action -> 0 [[-0.01572085 -0.41158791 -0.00485029  0.54902083]]
action -> 1 [[-0.02395261 -0.60664139  0.00613013  0.84017163]]
action -> 0 [[-0.03608544 -0.41160367  0.02293356  0.54942279]]
action -> 1 [[-0.04431751 -0.60704013  0.03392201  0.84924227]]
action -> 1 [[-0.05645831 -0.41239682  0.05090686  0.56741642]]
action -> 1 [[-0.06470625 -0.21802453  0.06225519  0.29119584]]
action -> 1 [[-0.06906674 -0.02384295  0.0680791   0.01877895]]
action -> 1 [[-0.0695436   0.17023997  0.06845468 -0.25167125]]
action -> 1 [[-0.0661388   0.36432102  0.06342126 -0.52200103]]
action -> 1 [[-0.05885238  0.55849564  0.05298124 -0.79404403]]
action -> 0 [[-0.04768247  0.75285187  0.03710036 -1.06960011]]
action -> 0 [[-0.03262543  0.55725943  0

action -> 1 [[ 0.03926254  0.21570032 -0.01437706 -0.31813186]]
action -> 0 [[ 0.04357655  0.41102406 -0.0207397  -0.61531389]]
action -> 1 [[ 0.05179703  0.21619793 -0.03304598 -0.32923459]]
action -> 1 [[ 0.05612099  0.41177435 -0.03963067 -0.63215279]]
action -> 0 [[ 0.06435648  0.60742615 -0.05227373 -0.93704842]]
action -> 0 [[ 0.076505    0.41304658 -0.07101469 -0.66123883]]
action -> 0 [[ 0.08476593  0.21898089 -0.08423947 -0.39173477]]
action -> 1 [[ 0.08914555  0.02514916 -0.09207417 -0.1267553 ]]
action -> 0 [[ 0.08964853  0.22146126 -0.09460927 -0.44700772]]
action -> 0 [[ 0.09407776  0.02779611 -0.10354943 -0.18558278]]
action -> 1 [[ 0.09463368 -0.16570358 -0.10726108  0.0727229 ]]
action -> 1 [[ 0.09131961  0.03077956 -0.10580662 -0.25178236]]
action -> 0 [[ 0.0919352   0.22724077 -0.11084227 -0.57587531]]
action -> 1 [[ 0.09648001  0.03383284 -0.12235978 -0.32006469]]
action -> 0 [[ 0.09715667  0.2304656  -0.12876107 -0.6486927 ]]
action -> 0 [[ 0.10176598  0.0373502  -0

KeyboardInterrupt: 